In [1]:
%pylab inline
import tensorflow as tf
import struct, gzip
import Image
from tensorflow.examples.tutorials.mnist import input_data

Populating the interactive namespace from numpy and matplotlib


In [2]:
data = genfromtxt('../data/exem/machine-error01.csv', skip_header=True, delimiter=',')
errors = data[:, 0]
data = data[:,2:]

train_errors = errors[:11000]
train_data = data[:11000, :]

test_errors = errors[11000:]
test_data = data[11000:, :]

In [3]:
# Parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 100
display_step = 1
n_steps = 6 * 60

# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 30 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float")

In [4]:
def next_batch(data, valid_data, n_sample=360, n_steps=n_steps):
    nrow, ncol = data.shape
    maximum = nrow - n_sample - n_steps
    idx = randint(0, maximum)
    batch_xs = data[idx:idx+n_sample,:]
    batch_y = valid_data[idx + n_sample + n_steps:idx + n_sample + n_steps + 1][0]
    return batch_xs, batch_y

while True:
    batch_xs, batch_y = next_batch(train_data, train_errors)
    print batch_xs, batch_y
    if batch_y == 1.:
        break


[[  39.   33.   23. ...,   48.  461.   78.]
 [  39.   33.   22. ...,   48.  461.   78.]
 [  39.   33.   23. ...,   48.  461.   78.]
 ..., 
 [  38.   33.   23. ...,   48.  461.   78.]
 [  38.   33.   23. ...,   48.  461.   78.]
 [  38.   33.   23. ...,   48.  461.   78.]] 1.0


In [24]:
weight_1 = tf.Variable(tf.random_normal([n_input, n_hidden_1]))
bias_1 = tf.Variable(tf.random_normal([n_hidden_1]))
layer_1 = tf.add(tf.matmul(x, weight_1), bias_1)
layer_1 = tf.nn.relu(layer_1)

weight_2 = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]))
bias_2 = tf.Variable(tf.random_normal([n_hidden_2]))
layer_2 = tf.add(tf.matmul(layer_1, weight_2), bias_2)
layer_2 = tf.nn.relu(layer_2)

weight_3 = tf.Variable(tf.random_normal([n_hidden_2, 1]))
layer_3 = tf.matmul(layer_2, weight_3)
layer_3 = tf.nn.relu(layer_3)


print x.get_shape()
print weight_1.get_shape()
print layer_1.get_shape()

(?, 30)
(30, 256)
(?, 256)


In [25]:
# Initializing the variables
init = tf.initialize_all_variables()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    batch_x, batch_y = next_batch(train_data, train_errors)
    print batch_x
    print sess.run([layer_3], feed_dict={x: batch_x, y: batch_y})

[[  32.    0.    0. ...,   48.  461.   78.]
 [  32.    0.    0. ...,   48.  461.   78.]
 [  32.    0.    0. ...,   48.  461.   78.]
 ..., 
 [  30.   38.   26. ...,   48.  461.   78.]
 [  30.   38.   27. ...,   48.  461.   78.]
 [  30.   38.   27. ...,   48.  461.   78.]]
[array([[ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],
       [ 0.],